In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unicodedata import normalize
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import GRU
from keras.layers import Dropout
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [46]:
def accent_remover(x):
    try:
        float(x)
        return x
    except:
        return normalize('NFKD',x).encode('ASCII', 'ignore').decode('ASCII')

In [47]:
# Importar dataset
dataset = pd.read_csv('datasets/datatran2017.csv', sep= ',', encoding='ISO-8859-1').dropna().drop_duplicates()
dataset = dataset.transform([accent_remover])

/home/maiconmares/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (1,2,3,4,7,8,9,10,11,12,13,14,15,16,17,28,29,30,31,32,33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [55]:
# Pega as dados das colunas:
# data_inversa	dia_semana	horario	uf	br	km	condicao_metereologica	latitude	longitude
X = dataset.iloc[:, 11:17].values
y = dataset.iloc[:, 22].values
y=np.array(y)
print(X)
print(y)

[['Com Vitimas Feridas' 'Plena Noite' 'Decrescente' 'Garoa/Chuvisco'
  'Simples' 'Reta']
 ['Com Vitimas Feridas' 'Plena Noite' 'Decrescente' 'Chuva' 'Simples'
  'Nao Informado']
 ['Com Vitimas Fatais' 'Plena Noite' 'Decrescente' 'Ceu Claro' 'Simples'
  'Curva']
 ...
 ['Com Vitimas Fatais' 'Anoitecer' 'Decrescente' 'Nublado' 'Simples'
  'Curva']
 ['Com Vitimas Feridas' 'Plena Noite' 'Crescente' 'Nublado' 'Simples'
  'Reta']
 ['Com Vitimas Feridas' 'Anoitecer' 'Crescente' 'Nublado' 'Dupla' 'Reta']]
[16  8  7 ... 30  8 13]


In [56]:
# Converte os dados de string para number gerando as categorias
encoder = LabelEncoder()
X[:,0] = encoder.fit_transform(X[:,0])
X[:,1] = encoder.fit_transform(X[:,1])
X[:,2] = encoder.fit_transform(X[:,2])
X[:,3] = encoder.fit_transform(X[:,3])
X[:,4] = encoder.fit_transform(X[:,4])
X[:,5] = encoder.fit_transform(X[:,5])

ct=ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0, 1, 2, 3, 4, 5])], remainder='passthrough')
X=np.array(ct.fit_transform(X))
print(X)

  (0, 1)	1.0
  (0, 5)	1.0
  (0, 8)	1.0
  (0, 11)	1.0
  (0, 20)	1.0
  (0, 26)	1.0
  (1, 1)	1.0
  (1, 5)	1.0
  (1, 8)	1.0
  (1, 10)	1.0
  (1, 20)	1.0
  (1, 24)	1.0
  (2, 0)	1.0
  (2, 5)	1.0
  (2, 8)	1.0
  (2, 9)	1.0
  (2, 20)	1.0
  (2, 21)	1.0
  (3, 1)	1.0
  (3, 5)	1.0
  (3, 8)	1.0
  (3, 9)	1.0
  (3, 20)	1.0
  (3, 26)	1.0
  (4, 0)	1.0
  :	:
  (44777, 26)	1.0
  (44778, 1)	1.0
  (44778, 6)	1.0
  (44778, 8)	1.0
  (44778, 9)	1.0
  (44778, 19)	1.0
  (44778, 26)	1.0
  (44779, 0)	1.0
  (44779, 4)	1.0
  (44779, 8)	1.0
  (44779, 15)	1.0
  (44779, 20)	1.0
  (44779, 21)	1.0
  (44780, 1)	1.0
  (44780, 5)	1.0
  (44780, 7)	1.0
  (44780, 15)	1.0
  (44780, 20)	1.0
  (44780, 26)	1.0
  (44781, 1)	1.0
  (44781, 4)	1.0
  (44781, 7)	1.0
  (44781, 15)	1.0
  (44781, 18)	1.0
  (44781, 26)	1.0


In [57]:
# Divide o dataset em conjunto de treino e conjunto de teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

TypeError: Singleton array array(<44782x31 sparse matrix of type '<class 'numpy.float64'>'
	with 268692 stored elements in Compressed Sparse Row format>,
      dtype=object) cannot be considered a valid collection.

In [42]:
# Converte os dados para uma mesma faixa de valores de 0 a 1
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
np.shape(X_train)

(35825, 19)

In [44]:
# Inicializa a ANN
regressor = Sequential()

# Adiciona a primeira camada
regressor.add(Dense(units=6, activation='relu'))
#regressor.add(Dropout(0.2))

# Segunda camada
regressor.add(Dense(units=6, activation='relu'))
#regressor.add(Dropout(0.2))

# Camada de saida
regressor.add(Dense(units=1, activation='sigmoid'))
#regressor.add(Dropout(0.2))

# Quarta camada
#regressor.add(GRU(units = 100))
#regressor.add(Dropout(0.2))

# Camada de saida
#regressor.add(Dense(units = 9))
2
# Compila
regressor.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=['accuracy'])

# Executa o treinamento
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)


Epoch 1/100
1120/1120 [==============================] - 1s 1ms/step - loss: -1658.1630 - accuracy: 0.0000e+00
Epoch 2/100
1120/1120 [==============================] - 1s 1ms/step - loss: -27379.8691 - accuracy: 0.0000e+00
Epoch 3/100
 394/1120 [=========>....................] - ETA: 0s - loss: -70865.7188 - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [49]:
# Executa predição
y_pred=regressor.predict(X_test)
y_pred

In [50]:
y_pred=(y_pred>0.5)
y_pred

In [51]:
from sklearn.metric import confusion_matrix
cm=confusion_matrix(y_test, y_pred)
print(cm)